In [20]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_ollama.llms import OllamaLLM

load_dotenv()

model = OllamaLLM(model='llama3.2:latest')

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善、專業的語調回覆。,生成文字時你不要出現(使用者名稱,團隊名稱,回信日期,感謝詞),因為我們會自已加入"),
        ("human","請請回覆這封客戶郵件：\n\n{email_content}")
    ]
)

def format_email_reply(reply:str):
    """格式化郵件回覆，添加專業的郵件格式"""
    formatted_reply = f"""
親愛的客戶,

    {reply}

    感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
---
回覆時間：{__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M')}
"""

    return formatted_reply.strip()

format_email = RunnableLambda(format_email_reply)

def add_quality_check(formatted_reply):
    """添加品質檢查和統計資訊"""
    word_count = len(formatted_reply.split())
    char_count = len(formatted_reply)

    quality_info = f"""
📊 回覆統計：
- 字數：{word_count} 字
- 字元數：{char_count} 字元
- 回覆品質：{'✅ 良好' if word_count >= 50 else '⚠️ 建議增加內容'}
{formatted_reply}
"""
    return quality_info

quality_check = RunnableLambda(add_quality_check)

chain = prompt_template | model | StrOutputParser() | format_email | quality_check 

In [22]:
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。

王小明
"""

result = chain.invoke({"email_content": customer_email})

# 輸出結果
print("=" * 60)
print("智能郵件回覆系統處理結果：")
print("=" * 60)
print(result)

智能郵件回覆系統處理結果：

📊 回覆統計：
- 字數：13 字
- 字元數：285 字元
- 回覆品質：⚠️ 建議增加內容
親愛的客戶,

    非常抱歉您遇到的問題！我們真心關心您的滿意度，讓我們一起解決這個問題 nhé！

根據您的描述，我們會盡量幫助您退貨或換貨。由於您購買的產品已經一週了，所以我們需要確認是否有可能進行退貨或換貨。

請給我們一些時間，並且將包裝損壞的證據（例如照相或照片）寄送至我們的客服信箱。然後，我們將根據公司的政策進行評估，是否可以進行退貨或換貨。

如果您有任何其他問題或需求，或者需要進一步協助，请在下一次回覆中告知我們 nhé！

    感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
---
回覆時間：2025-10-06 14:22

